<a href="https://colab.research.google.com/github/alexjercan/object-detection/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install albumentations==0.4.6

In [ ]:
# git clone
!git clone https://github.com/alexjercan/object-detection
%cd object-detection

import torch

clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
# download dataset
torch.hub.download_url_to_file('https://github.com/alexjercan/object-detection/releases/download/v1.0/bdataset108.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../ && rm tmp.zip

In [ ]:
import cv2
import torch
import torch.optim as optim
import config

from dataset import create_dataloader
from general import train, test
from model import Model
from tqdm import tqdm
from common import L_RGB, L_DEPTH, L_NORMAL, build_targets, save_checkpoint, load_checkpoint, count_channles, load_yaml
from loss import LossFunction

In [ ]:
# config
scaled_anchors = (
    torch.tensor(config.ANCHORS)
    * torch.tensor(config.S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
).to(config.DEVICE)

config.DATASET = "bdataset108"
config.LOAD_MODEL = False

torch.backends.cudnn.benchmark = True

In [ ]:
# create model
model = Model(model_dict=load_yaml(config.MODEL_DICT),
                  in_channels=count_channles(config.LAYERS),
                  num_classes=config.NUM_CLASSES
                  ).to(config.DEVICE)
optimizer = optim.Adam(model.parameters(),
                        lr=config.LEARNING_RATE,
                        weight_decay=config.WEIGHT_DECAY)
loss_fn = LossFunction()
scaler = torch.cuda.amp.GradScaler()

if config.LOAD_MODEL:
    load_checkpoint(config.CHECKPOINT_FILE, model, optimizer, config.LEARNING_RATE, config.DEVICE)

In [ ]:
# create train dataloader
dataset, loader = create_dataloader(config.IMG_DIR + "/train", config.LABEL_DIR + "/train", image_size=config.IMAGE_SIZE,
                                    batch_size=config.BATCH_SIZE, S=config.S, anchors=config.ANCHORS,
                                    transform=None, used_layers=config.LAYERS)

In [ ]:
# train model
for epoch in range(config.NUM_EPOCHS):
    train(loader, model, optimizer, loss_fn, scaler, scaled_anchors)

In [ ]:
# create test dataloader
dataset, loader = create_dataloader(config.IMG_DIR + "/test", config.LABEL_DIR + "/test", image_size=config.IMAGE_SIZE,
                                    batch_size=config.BATCH_SIZE, S=config.S, anchors=config.ANCHORS,
                                    transform=None, used_layers=config.LAYERS)

In [ ]:
# test model
test(loader, model)